In [1]:
from datetime import date, timedelta
from contextlib import contextmanager
import time
import os

import numpy as np
import pandas as pd


IDENT_BASE = "ABC"


def gen_identifiers(n=3000):
    return [IDENT_BASE + str(x).zfill(6)
            for x in range(n)]


def gen_dates(n=3000):
    today = date.today()
    start = today - timedelta(days=n)
    return pd.date_range(start, today)


def gen_data():
    ids = gen_identifiers()
    dates = gen_dates()
    data = np.random.random(len(ids))

    results = []
    for date in dates:
        df = pd.DataFrame({
            'identifier': ids,
            'value': data
        })
        df['date'] = date
        results.append(df)

    return pd.concat(results)

df = gen_data()
print(f'DataFrame: {df.shape}')

DataFrame: (9003000, 3)


In [2]:
def test_and_benchmark(func_to_benchmark, file_name, func_to_read, is_h5=False, *args, **kwargs):
    """
    Test and benchmark a given algorithm to compress a dataframe.
    """
    start = time.perf_counter()
    func_to_benchmark(file_name, *args, **kwargs)
    save_time = time.perf_counter() - start
    
    start = time.perf_counter()

    if is_h5:
        func_to_read(file_name, key='df')
    else:
        func_to_read(file_name)

    load_time = time.perf_counter() - start
    
    size_in_bytes = os.path.getsize(file_name)
    
    res = {
        'file_name': file_name,
        'save_time': save_time,
        'load_time': load_time,
        'file_size': size_in_bytes
    }
    
    print(res)

    return res

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9003000 entries, 0 to 2999
Data columns (total 3 columns):
identifier    object
value         float64
date          datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 274.7+ MB


In [4]:
h_compression_engines = [
    'zlib', 'lzo', 'bzip2', 'blosc',
    'blosc:blosclz', 'blosc:lz4', 'blosc:lz4hc',
    'blosc:snappy', 'blosc:zlib', 'blosc:zstd'
]

records = []
for comp_engine in h_compression_engines:
    for c_level in range(1, 10):
        result = test_and_benchmark(
            df.to_hdf,
            f'lvl{c_level}_{comp_engine}_initial.h5',
            pd.read_hdf,
            complib=comp_engine,
            complevel=c_level,
            key='df',
            is_h5=True
        )
        records.append(result)

for parq_engine in ['snappy', 'gzip', 'brotli']:
    res = test_and_benchmark(
        df.to_parquet,
        f'{parq_engine}_initial.parquet',
        pd.read_parquet,
        compression=parq_engine,
    )
    records.append(res)

{'file_name': 'lvl1_zlib_initial.h5', 'save_time': 1.9251981999999999, 'load_time': 1.1804757999999997, 'file_size': 128683644}
{'file_name': 'lvl2_zlib_initial.h5', 'save_time': 1.5262379, 'load_time': 1.2209019999999997, 'file_size': 128683603}
{'file_name': 'lvl3_zlib_initial.h5', 'save_time': 1.5083728, 'load_time': 1.2270727, 'file_size': 128683459}
{'file_name': 'lvl4_zlib_initial.h5', 'save_time': 1.9430028999999998, 'load_time': 1.300381999999999, 'file_size': 128389366}
{'file_name': 'lvl5_zlib_initial.h5', 'save_time': 1.9347189, 'load_time': 1.2645329999999984, 'file_size': 128403136}
{'file_name': 'lvl6_zlib_initial.h5', 'save_time': 1.9884337999999993, 'load_time': 1.2461622999999982, 'file_size': 128399712}
{'file_name': 'lvl7_zlib_initial.h5', 'save_time': 1.9756, 'load_time': 1.2566148999999989, 'file_size': 128372460}
{'file_name': 'lvl8_zlib_initial.h5', 'save_time': 2.011139100000001, 'load_time': 1.2440635000000029, 'file_size': 128320512}
{'file_name': 'lvl9_zlib_i

{'file_name': 'lvl2_blosc:snappy_initial.h5', 'save_time': 1.0346236999999974, 'load_time': 1.0896160000000066, 'file_size': 145280253}
{'file_name': 'lvl3_blosc:snappy_initial.h5', 'save_time': 0.9885836999999924, 'load_time': 1.0724102000000073, 'file_size': 145280253}
{'file_name': 'lvl4_blosc:snappy_initial.h5', 'save_time': 1.1660128000000043, 'load_time': 1.0799239000000114, 'file_size': 145280253}
{'file_name': 'lvl5_blosc:snappy_initial.h5', 'save_time': 0.9939644000000101, 'load_time': 1.0625943999999947, 'file_size': 145280253}
{'file_name': 'lvl6_blosc:snappy_initial.h5', 'save_time': 1.1859230000000025, 'load_time': 1.0543846999999857, 'file_size': 145280253}
{'file_name': 'lvl7_blosc:snappy_initial.h5', 'save_time': 1.0029427000000055, 'load_time': 1.0639255000000105, 'file_size': 145280253}
{'file_name': 'lvl8_blosc:snappy_initial.h5', 'save_time': 1.0692114000000004, 'load_time': 1.054091600000021, 'file_size': 145280253}
{'file_name': 'lvl9_blosc:snappy_initial.h5', 'sa

In [5]:
optimised_df = df.copy()
optimised_df['value'] = optimised_df['value'].astype('float32')
optimised_df['identifier'] = optimised_df['identifier'].astype('category')
optimised_df['date'] = optimised_df['value'].astype('category')

In [6]:
optimised_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9003000 entries, 0 to 2999
Data columns (total 3 columns):
identifier    category
value         float32
date          category
dtypes: category(2), float32(1)
memory usage: 137.6 MB


In [7]:
for comp_engine in h_compression_engines:
    for c_level in range(1, 10):
        result = test_and_benchmark(
            df.to_hdf,
            f'lvl{c_level}_{comp_engine}_optimised.h5',
            pd.read_hdf,
            complib=comp_engine,
            complevel=c_level,
            key='df',
            is_h5=True
        )
        records.append(result)

{'file_name': 'lvl1_zlib_optimised.h5', 'save_time': 1.6550492000000077, 'load_time': 1.170646300000044, 'file_size': 171790243}
{'file_name': 'lvl2_zlib_optimised.h5', 'save_time': 2.182313899999997, 'load_time': 1.1663815999999656, 'file_size': 171790208}
{'file_name': 'lvl3_zlib_optimised.h5', 'save_time': 1.539382900000021, 'load_time': 1.1755056999999738, 'file_size': 171790068}
{'file_name': 'lvl4_zlib_optimised.h5', 'save_time': 1.976733500000023, 'load_time': 1.251539800000046, 'file_size': 171502456}
{'file_name': 'lvl5_zlib_optimised.h5', 'save_time': 1.9802776000000222, 'load_time': 1.245097200000032, 'file_size': 171515864}
{'file_name': 'lvl6_zlib_optimised.h5', 'save_time': 2.090701499999966, 'load_time': 1.255020400000035, 'file_size': 171512552}
{'file_name': 'lvl7_zlib_optimised.h5', 'save_time': 2.1953942000000097, 'load_time': 1.2575075999999967, 'file_size': 171486018}
{'file_name': 'lvl8_zlib_optimised.h5', 'save_time': 2.3257615999999643, 'load_time': 1.2520652000

{'file_name': 'lvl1_blosc:snappy_optimised.h5', 'save_time': 1.6688656000000037, 'load_time': 1.0653891999999132, 'file_size': 188234398}
{'file_name': 'lvl2_blosc:snappy_optimised.h5', 'save_time': 2.456196699999964, 'load_time': 1.0869629000000032, 'file_size': 188234398}
{'file_name': 'lvl3_blosc:snappy_optimised.h5', 'save_time': 1.288898099999983, 'load_time': 1.0639520999999377, 'file_size': 188234398}
{'file_name': 'lvl4_blosc:snappy_optimised.h5', 'save_time': 1.4712614999999687, 'load_time': 1.0640165000000934, 'file_size': 188234398}
{'file_name': 'lvl5_blosc:snappy_optimised.h5', 'save_time': 1.4030338000000029, 'load_time': 1.0528432999999495, 'file_size': 188234398}
{'file_name': 'lvl6_blosc:snappy_optimised.h5', 'save_time': 1.8123690999999553, 'load_time': 1.0678437999999915, 'file_size': 188234398}
{'file_name': 'lvl7_blosc:snappy_optimised.h5', 'save_time': 1.181080800000018, 'load_time': 1.0652630000000727, 'file_size': 188234398}
{'file_name': 'lvl8_blosc:snappy_opti

In [8]:
result_df = pd.DataFrame.from_records(records, index='file_name')
result_df.head()

,file_size,load_time,save_time
file_name,,,
lvl1_zlib_initial.h5,128683644,1.180476,1.925198
lvl2_zlib_initial.h5,128683603,1.220902,1.526238
lvl3_zlib_initial.h5,128683459,1.227073,1.508373
lvl4_zlib_initial.h5,128389366,1.300382,1.943003
lvl5_zlib_initial.h5,128403136,1.264533,1.934719


In [9]:
df.to_csv('no_compression.csv')

In [10]:
no_comp_filesize = os.path.getsize('no_compression.csv')

In [11]:
result_df['compression_ratio'] = no_comp_filesize / result_df['file_size']

In [12]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(
    tools=TOOLS,
    tooltips=[
        ("File", "@file_name"),
        ("load_time", "$x"),
        ("compression_ratio", "$y"),
    ],
    y_axis_type="log",
    title="Effects of compression on load time",
    y_axis_label='Compression Ratio vs CSV',
    x_axis_label='Time to decompress (s)',
)

p.scatter(
    source=result_df,
    x='load_time',
    y='compression_ratio',
    marker='circle'
)
show(p)

Loading BokehJS ...

In [13]:
p = figure(
    tools=TOOLS,
    tooltips=[
        ("File", "@file_name"),
        ("Decompression (s)", "$x"),
        ("Compression (s)", "$y"),
    ],
    title="Association between compression and decompression time",
    y_axis_label='Time to compress (s)',
    x_axis_label='Time to decompress (s)',
)

p.scatter(
    source=result_df,
    x='load_time',
    y='save_time',
    marker='circle'
)
show(p)

In [17]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(result_df)

                                 file_size  load_time  save_time  \
file_name                                                          
lvl1_zlib_initial.h5             128683644   1.180476   1.925198   
lvl2_zlib_initial.h5             128683603   1.220902   1.526238   
lvl3_zlib_initial.h5             128683459   1.227073   1.508373   
lvl4_zlib_initial.h5             128389366   1.300382   1.943003   
lvl5_zlib_initial.h5             128403136   1.264533   1.934719   
lvl6_zlib_initial.h5             128399712   1.246162   1.988434   
lvl7_zlib_initial.h5             128372460   1.256615   1.975600   
lvl8_zlib_initial.h5             128320512   1.244064   2.011139   
lvl9_zlib_initial.h5             128319923   1.235851   1.925217   
lvl1_lzo_initial.h5              140190352   1.117739   1.011438   
lvl2_lzo_initial.h5              140190352   1.120660   0.934485   
lvl3_lzo_initial.h5              140190352   1.123009   0.968235   
lvl4_lzo_initial.h5              140190352   1.1

In [20]:
parq = pd.read_parquet('brotli_initial.parquet')
pd.testing.assert_frame_equal(parq, df)